In [139]:
import numpy as np
import cv2

In [140]:
# read depth image
depth_scale = 0.00012498664727900177
depth_img = cv2.imread('depth.png')
dpt = depth_img[:, :, 2] + depth_img[:, :, 1] * 256
dpt = dpt * depth_scale

# read seg image
seg = cv2.imread('seg.png')[...,0]  # 255: fore ground, 0: background

# read intrinsics and extrinsics
K = np.load('intrinsic.npy')

# print(dpt.shape, depth_img.shape)
print(K)

[[415.69219382   0.         320.        ]
 [  0.         415.69219382 240.        ]
 [  0.           0.           1.        ]]


In [141]:
# task1: convert depth image to point cloud
def depth2pc(depth, seg, K):
    alpha, _, c_x = K[0]
    _, beta, c_y = K[1]
    
    coordinate = np.where(seg != 0)
    z = depth[coordinate]
    u, v = coordinate[0], coordinate[1]
    x = (u - c_x) * z / alpha
    y = (v - c_y) * z / beta
    # print(x.shape)
    n = x.shape[0]
    pc = np.concatenate((x,y,z), axis=0).reshape(n, 3)
    return pc

partial_pc = depth2pc(dpt, seg, K)

# For debug and submission
np.savetxt('../results/pc_from_depth.txt', partial_pc)

In [142]:
# task2: compute one-way chamfer distance to the complete shape
full_pc = np.loadtxt('aligned_full_pc.txt')
print(full_pc.shape, partial_pc.shape)

def random_sample(pc, num):
    permu = np.random.permutation(pc.shape[0])
    return pc[permu][:num]

partial_pc_sampled = random_sample(partial_pc, 2048)
full_pc_sampled = random_sample(full_pc, 2048)

def one_way_CD(P, Q):
    dist_matrix = np.linalg.norm(Q[:, np.newaxis, :] - P, axis = 2)
    min_dist = np.min(dist_matrix, axis = 1)
    sum_min_dist = np.sum(min_dist)
    return sum_min_dist/P.shape[0]  
# -----------TODO---------------
# implement one way chamfer distance
# -----------TODO---------------

# one_way_CD = one_way_CD(partial_pc, full_pc)
one_way_CD = one_way_CD(partial_pc_sampled, full_pc_sampled)
print('one way chamfer distance: ', one_way_CD)

# For submission
np.savetxt('../results/one_way_CD.txt', [one_way_CD])

(6858, 3) (19375, 3)
one way chamfer distance:  1.105627613046914
